# Notebook 8: Reducing Hallucinations

In this notebook, you'll learn techniques to get more reliable, grounded responses and reduce made-up information.

## What You'll Learn

- What hallucinations are and why they happen
- Grounding responses with context
- Encouraging "I don't know" responses
- Verification techniques (citations, quotes)

## Reference

- [Mistral Prompting Documentation](https://docs.mistral.ai/guides/prompting/)

---
## Setup

In [ ]:
%run 00_setup.ipynb

---
## Section 1: What Are Hallucinations?

**Hallucinations** occur when a model generates plausible-sounding but factually incorrect information.

Examples:
- Inventing fake citations or papers
- Making up statistics or dates
- Confidently stating incorrect facts
- Fabricating quotes or events

**Why they happen:**
- Models are trained to produce fluent, confident text
- No built-in "I don't know" instinct
- Models fill gaps with plausible-sounding content
- Ambiguous prompts invite gap-filling

In [ ]:
# Example: Asking about something the model might hallucinate on
# Note: This is demonstrating potential hallucination - the info may be inaccurate

risky_prompt = """What were the exact revenue figures for Acme Corp in Q3 2024?"""

print("RISKY PROMPT (no grounding):")
print("-" * 40)
response = call_mistral(user_prompt=risky_prompt, temperature=0.7)
print(response)
print("\n⚠️ Warning: The model may have made up these numbers!")

---
## Section 2: Why Hallucinations Happen

Models hallucinate because:

1. **Confidence by design** - Trained to generate fluent, confident responses
2. **No knowledge cutoff awareness** - May not know when information is outdated
3. **Pattern completion** - Fill in gaps with statistically likely content
4. **Open-ended prompts** - Vague questions invite fabrication

**High-risk scenarios:**
- Recent events (after training cutoff)
- Specific statistics, dates, or figures
- Obscure or niche topics
- Anything requiring real-time data

---
## Section 3: Grounding with Context

The most effective way to reduce hallucinations: **provide source material**.

```
Answer the question based ONLY on the context provided.

<context>
[Your source document]
</context>

Question: [Your question]
```

In [ ]:
# Grounding with context
context = """
TechCorp Q3 2024 Financial Report Summary:
- Total Revenue: $4.2 billion (up 12% YoY)
- Operating Income: $890 million
- Cloud Services Revenue: $1.8 billion
- Employee Count: 15,400
- CEO Statement: "We exceeded expectations in cloud services."
"""

grounded_prompt = f"""Answer the question based ONLY on the context provided.
If the information is not in the context, say "This information is not provided in the context."

<context>
{context}
</context>

Question: What was TechCorp's cloud services revenue in Q3 2024?"""

print("GROUNDED RESPONSE:")
print("-" * 40)
response = call_mistral(user_prompt=grounded_prompt, temperature=0)
print(response)

In [ ]:
# Testing with a question NOT in the context
out_of_context_prompt = f"""Answer the question based ONLY on the context provided.
If the information is not in the context, say "This information is not provided in the context."

<context>
{context}
</context>

Question: What was TechCorp's marketing budget in Q3 2024?"""

print("QUESTION NOT IN CONTEXT:")
print("-" * 40)
response = call_mistral(user_prompt=out_of_context_prompt, temperature=0)
print(response)

---
## Section 4: Encouraging "I Don't Know"

Explicitly give the model permission to admit uncertainty.

**Key phrases:**
- "If you don't know, say 'I don't know'"
- "If the answer is not in the context, say so"
- "Do not make up information"
- "It's okay to say you're unsure"

In [ ]:
# System prompt that encourages honesty about uncertainty
honest_system_prompt = """You are a helpful assistant that values accuracy over completeness.

Important rules:
- Only provide information you are confident about
- If you're unsure or the information might be outdated, say so
- Never make up statistics, dates, or specific figures
- It's better to say "I don't know" than to provide incorrect information"""

question = "What was Apple's exact stock price at market close yesterday?"

print("WITH HONESTY INSTRUCTIONS:")
print("-" * 40)
response = call_mistral(
    system_prompt=honest_system_prompt,
    user_prompt=question,
    temperature=0
)
print(response)

In [ ]:
# Contrast: Without honesty instructions
print("WITHOUT HONESTY INSTRUCTIONS:")
print("-" * 40)
response_risky = call_mistral(
    user_prompt=question,
    temperature=0
)
print(response_risky)

---
## Section 5: Verification Techniques

Make the model's reasoning verifiable:

### 1. Require Citations/Quotes
Ask the model to quote the exact text that supports its answer.

### 2. Request Confidence Levels
Ask for a confidence rating with the answer.

### 3. Evidence-First Format
Quote first, then answer.

In [ ]:
# Requiring quotes/citations
document = """
Climate Change Report 2024:

Global temperatures have risen by 1.2°C since pre-industrial times. The Arctic 
is warming twice as fast as the global average. Sea levels have risen by 8 inches 
since 1900. Scientists project that without intervention, temperatures could rise 
by an additional 1.5-4.5°C by 2100.

Key recommendations include reducing carbon emissions by 45% by 2030 and achieving 
net-zero emissions by 2050.
"""

citation_prompt = f"""Answer the question based on the document below.

IMPORTANT: For each claim in your answer:
1. Quote the exact text from the document that supports it
2. If no supporting text exists, say "Not stated in document"

<document>
{document}
</document>

Question: How much have global temperatures risen, and what are the projections for the future?

Answer with citations:"""

response = call_mistral(user_prompt=citation_prompt, temperature=0)
print(response)

In [ ]:
# Evidence-first format
evidence_first_prompt = f"""Answer the question using ONLY the provided document.

Format your response as:
<evidence>
[Quote the relevant passages from the document]
</evidence>

<answer>
[Your answer based only on the evidence above]
</answer>

<confidence>
[High/Medium/Low - based on how directly the evidence supports the answer]
</confidence>

<document>
{document}
</document>

Question: What specific emission reduction targets are mentioned?"""

response = call_mistral(user_prompt=evidence_first_prompt, temperature=0)
print(response)

---
## Section 6: What NOT to Do (Negative Examples)

### ❌ Open-ended without grounding
```
Tell me about the company's Q3 performance.
```
No source material = invitation to fabricate.

### ❌ Assuming current knowledge
```
What's the current price of Bitcoin?
```
Model doesn't have real-time data.

### ❌ Punishing uncertainty
```
You must provide an answer. Do not say you don't know.
```
Forces hallucination.

### ❌ Asking for specific unverifiable details
```
Give me the exact number of...
```
Without source, encourages fabrication.

In [ ]:
# Demonstrating the danger of "must answer" instructions
print("DANGEROUS - Forced answer (may hallucinate):")
print("-" * 40)
forced_prompt = """What is the exact population of Springfield, Illinois as of today?
You must provide a specific number. Do not say you don't know."""

response = call_mistral(user_prompt=forced_prompt, temperature=0)
print(response)
print("\n⚠️ This number may be fabricated!")

print("\n" + "=" * 50 + "\n")

# Better approach
print("BETTER - Allows uncertainty:")
print("-" * 40)
better_prompt = """What is the population of Springfield, Illinois?
If you don't have current data, provide your best estimate and note the uncertainty."""

response = call_mistral(user_prompt=better_prompt, temperature=0)
print(response)

---
## Exercise 1: With vs Without Context

Compare responses when grounding with context vs without.

In [ ]:
# Product specification document
product_spec = """
Product: XPhone Pro Max
Display: 6.7" OLED, 2778 x 1284 resolution
Processor: A17 Bionic chip
Storage Options: 128GB, 256GB, 512GB, 1TB
Battery: 4422 mAh, up to 29 hours video playback
Camera: 48MP main, 12MP ultra-wide, 12MP telephoto
Price: Starting at $1,099
Colors: Natural Titanium, Blue Titanium, White Titanium, Black Titanium
"""

question = "What are the camera specifications of the XPhone Pro Max?"

# Without context (risky)
print("WITHOUT CONTEXT (potential hallucination):")
print("-" * 40)
response_no_context = call_mistral(user_prompt=question, temperature=0)
print(response_no_context)

print("\n" + "=" * 50 + "\n")

# With context (grounded)
print("WITH CONTEXT (grounded):")
print("-" * 40)
grounded = f"""Answer based ONLY on this product specification:

<spec>
{product_spec}
</spec>

Question: {question}"""
response_with_context = call_mistral(user_prompt=grounded, temperature=0)
print(response_with_context)

---
## Exercise 2: Building an "Honest" Prompt

Create a Q&A prompt that grounds, quotes, and admits uncertainty.

In [ ]:
# Document to use
company_faq = """
Acme Inc FAQ:

Q: When was Acme Inc founded?
A: Acme Inc was founded in 1995 by Jane Smith in Austin, Texas.

Q: What products does Acme sell?
A: We sell cloud computing solutions, data analytics tools, and AI services.

Q: How many employees does Acme have?
A: As of 2023, we employ over 5,000 people worldwide.

Q: What are your office locations?
A: We have offices in Austin (HQ), San Francisco, New York, London, and Singapore.
"""

# Test questions (some answerable, some not)
test_questions = [
    "Who founded Acme Inc?",  # In context
    "What is Acme's annual revenue?",  # NOT in context
    "Where is Acme's headquarters?",  # In context
    "Does Acme offer health insurance to employees?"  # NOT in context
]

# TODO: Create a robust prompt template that:
# 1. Grounds answers in the provided context
# 2. Quotes supporting text
# 3. Says "Not found in context" when information isn't available

honest_qa_template = f"""Answer questions using ONLY the FAQ document below.

Rules:
- Quote the relevant text that supports your answer
- If the information is not in the FAQ, respond: "This information is not available in the FAQ."
- Do not make assumptions or add information not in the document

<faq>
{company_faq}
</faq>

Question: {{question}}

Answer:"""

for q in test_questions:
    prompt = honest_qa_template.format(question=q)
    response = call_mistral(user_prompt=prompt, temperature=0)
    print(f"Q: {q}")
    print(f"A: {response}")
    print("-" * 50)

---
## Exercise 3: Citation Enforcement

Build a prompt that requires citations and verify they exist.

In [ ]:
# Research summary to work with
research_summary = """
Study: Effects of Remote Work on Productivity (2024)

Key Findings:
1. Remote workers reported 23% higher job satisfaction compared to office workers.
2. Productivity metrics showed a 13% increase in output for remote workers.
3. However, collaboration scores decreased by 18% in fully remote teams.
4. Hybrid workers (3 days office, 2 days remote) showed the best overall outcomes.
5. The study surveyed 10,000 employees across 50 companies in the tech sector.

Limitations:
- Study focused only on tech sector, may not generalize to other industries.
- Self-reported productivity measures may have bias.
"""

# TODO: Create a prompt that:
# 1. Answers a question about the research
# 2. Requires exact quotes as citations
# 3. Returns citations in a verifiable format

citation_template = """
# Your citation-requiring prompt here
"""

# Then verify the citations exist in the source text
# Hint: You can check if quoted text appears in research_summary

---
## Key Takeaways

1. **Always ground with source context** when accuracy matters

2. **Explicitly permit "I don't know"** - Remove pressure to always have an answer

3. **Require citations/quotes** for verifiability

4. **Never force answers** - "You must answer" leads to hallucination

5. **Be aware of limitations** - No real-time data, knowledge cutoff exists

---

## Next Steps

Now that you can reduce hallucinations, let's put all the techniques together in real-world scenarios.

📚 [Continue to Notebook 9: Putting It All Together →](09_putting_it_all_together.ipynb)